<a href="https://colab.research.google.com/github/Falconwatch/cybersec_ht/blob/main/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q fuzzywuzzy
!pip install -q python-Levenshtein

import lightgbm as lgbm
import joblib
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import fuzz
import rapidfuzz

In [4]:
examples = pd.read_excel("examples.xlsx")
cust_1 = pd.read_excel("cust_1.xlsx")
cust_2 = pd.read_excel("cust_2.xlsx")

In [39]:
class HotelComparator():
  def __init__(self, path):
    """
    param path: Путь до сохранённой модели
    """
    self.__clf = joblib.load('hotel_clf.pkl')
    self.__feature_fields = ['hotel_name', 'city_name', 'hotel_address', 'star_rating', 'postal_code']
    self.__th = 0.0002

  def similarity_ratio_func(self, df, field):
    #print("in similarity_ratio_func")
    def string_similarity(s1,s2):
      #print("In string_similarity")
      return rapidfuzz.fuzz_cpp.WRatio(s1,s2)/100
      return fuzz.WRatio(s1, s2)/100

    def postalcode_similarity(s1, s2):
      longest_string = max([len(s1), len(s2)])
      s1 = s1+"0"*(longest_string - len(s1))
      s2 = s2+"0"*(longest_string - len(s2))
      t = [a==b for a,b in zip(s1, s2)][::-1]
      max_possible_similarity = sum([10**i for i in range(longest_string)])
      current_similarity = sum([t[i]*10**i for i in range(len(t))])
      difference = max_possible_similarity - current_similarity
      difference_ratio = difference/max_possible_similarity
      return 1-difference_ratio
    
    def rating_star_diff(s1, s2):
      try:
        r = abs(float(s1)- float(s2))
        return r
      except:
        return None

    def universal_wrapper(x1, x2, func):
      if pd.isna(x1) or pd.isna(x2):
        return None
      x1=str(x1).upper()
      x2=str(x2).upper()
      return func(x1,x2)

    name1 = "c1." + field
    name2 = "c2." + field
    name3 = field + "_similarity"
    
    if "postal_code" in field:
      #print("procees postal code")
      df[name3] = df[[name1, name2]].apply(lambda x: universal_wrapper(x[name1], x[name2], postalcode_similarity),
                                            axis=1)
    elif "star_rating" in field:
      #print("procees star_rating")
      df[name3] = df[[name1, name2]].apply(lambda x: universal_wrapper(x[name1], x[name2], rating_star_diff),
                                            axis=1)
    else:
      #print("Process:", name3)
      df[name3] = df[[name1, name2]].apply(lambda x: universal_wrapper(x[name1], x[name2], string_similarity),
                                            axis=1)

  def generate_features(self, df):
    #print("in generate_features")
    #print(self.__feature_fields)
    for ff in tqdm(self.__feature_fields):
      self.similarity_ratio_func(df, ff)

  
  def compare_hotels_lists(self, hotels_1, hotels_2,
                           country_code_name_1 = "c1.country_code",
                           country_code_name_2 = "c2.country_code",):
    """
    param hotels_1: pandas dataframe со списком отелей от поставщика 1
    param hotels_2: pandas dataframe со списком отелей от поставщика 2
    """
    hotels_1 = hotels_1.rename({country_code_name_1:"country_code"}, axis=1)
    hotels_2 = hotels_2.rename({country_code_name_2:"country_code"}, axis=1)
    countries = hotels_1["country_code"].unique()

    country_datas = list()
    for country in countries:
      h1 = hotels_1[hotels_1["country_code"] == country]
      h2 = hotels_2[hotels_2["country_code"] == country]
      total = h1.reset_index(drop=True).merge(h2.reset_index(drop=True),
                                              on =["country_code"])
      country_datas.append(total)

    features = [f+"_similarity" for f in self.__feature_fields]
    full_data = pd.concat(country_datas)
    self.generate_features(full_data)
    probas = self.__clf.predict_proba(full_data[features])[:,1]
    predictions = (probas > self.__th).astype(int)

    full_data["match"] = predictions
    result = full_data.loc[full_data["match"]==1, ["c1.key", "c2.key"]]

    return result


In [40]:
hc = HotelComparator("hotel_clf.pkl")

In [43]:
t = hc.compare_hotels_lists(cust_1.sample(5000), cust_2.sample(5000))

100%|██████████| 5/5 [02:02<00:00, 24.43s/it]
